This notebook provides examples to go along with the [textbook](http://underactuated.csail.mit.edu/lqr.html).  I recommend having both windows open, side-by-side!

[Click here](http://underactuated.csail.mit.edu/drake.html#notebooks) for instructions on how to run the notebook on Deepnote and/or Google Colab.

In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pydot
from IPython.display import SVG, Markdown, clear_output, display
from pydrake.all import (AddMultibodyPlantSceneGraph, ConstantVectorSource,
                         ControllabilityMatrix, DiagramBuilder,
                         DiscreteAlgebraicRiccatiEquation,
                         DiscreteTimeLinearQuadraticRegulator, IsControllable,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MatrixGain, MeshcatVisualizer, OutputPortSelection,
                         Parser, Simulator, StartMeshcat, ToLatex)
from pydrake.common.containers import namedview

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()


In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# Discrete-time vs continuous-time LQR

Let's compare the two solutions in our simplest of systems: the double integrator.

## Continuous time

In [ ]:
def ct_demo():
    A = np.array([[0,1],[0,0]])
    B = np.array([[0],[1]])
    Q = np.identity(2)
    R = np.identity(1)

    K,S = LinearQuadraticRegulator(A,B,Q,R)
    display(Markdown(f"$S = {ToLatex(S)}$\n"))

    eigenvalues, eigenvectors = np.linalg.eig(S)
    display(Markdown(f"$eig(S) = {ToLatex(eigenvalues)}$\n"))

ct_demo()

## Discrete time

In [ ]:
def dt_demo(h=1):
    A = np.array([[1,h],[0,1]])
    B = np.array([[0],[h]])
    Q = h*np.identity(2)
    R = h*np.identity(1)

    K,S = DiscreteTimeLinearQuadraticRegulator(A,B,Q,R)
    display(Markdown(f"$S = {ToLatex(S)}$\n"))
    eigenvalues, eigenvectors = np.linalg.eig(S)
    display(Markdown(f"$eig(S) = {ToLatex(eigenvalues)}$\n"))

dt_demo(h=1)

There are a few things to notice here.  First, the discrete-time solution converges to the continuous-time solution as the timestep goes to zero.  Second, the cost-to-go is always higher in the discrete-time version: you should think of the discrete time as adding an additional constraint that the control decision can only be changed once per timestep.  Adding constraints can only increase the total cost.

### Side note: Algebraic solution

In the DP chapter, I was able to give a nice closed-form solution for the continuous-time case:$$S = \begin{bmatrix} \sqrt{3} & 1 \\ 1 & \sqrt{3} \end{bmatrix}.$$  The discrete-time case is not as clean.  Even when $h=1$, the discrete-time Riccati equation, using $$S = \begin{bmatrix} a & b \\ b & c \end{bmatrix},$$ results in three equations: \begin{gather*} b^2 = 1+c \\ 1 + c + bc = a + ac \\ c^2 = 2b + a + ac. \end{gather*}  With a little work, you can reduce this to to a quadratic equation in $b$, with a solution, $b = \frac{1}{4}(1 + \sqrt{21} + \sqrt{2(3+\sqrt{21})}).$  Not so nice! 

### Side note: Exact integration

One can also use the exact integral of the linear system $e^Ah$ in the discretization, instead of the Euler discretization.  It doesn't change the basic observation.

# LQR via Fitted Value Iteration

Double integrator.  Discrete-time, infinite-horizon, discounted.  This is the "traditional" fitted value iteration, $J=x^TSx$ as the (linear) function approximator.  Is samples both $x$ and $u$, and takes an argmin over the samples $u$.

In [ ]:
# Define the double integrator
timestep = 0.1
A = np.eye(2) + timestep*np.array([[0., 1.], [0., 0.]])
B = timestep*np.array([[0.], [1.]])
Q = timestep*np.eye(2)
R = timestep*np.eye(1)

def quadratic_regulator_cost(x, u):
    return (x * (Q @ x)).sum(axis=0) + (u * (R @ u)).sum(axis=0)

def FittedValueIteration(S_guess, gamma=0.9):

    S_optimal = DiscreteAlgebraicRiccatiEquation(A=np.sqrt(gamma) * A,
                                                 B=B,
                                                 Q=Q,
                                                 R=R / gamma)

    x1s = np.linspace(-5,5,31)
    x2s = np.linspace(-4,4,31)
    us = np.linspace(-1,1,9)
    Us, X1s, X2s = np.meshgrid(us, x1s, x2s, indexing='ij')
    XwithU = np.vstack((X1s.flatten(), X2s.flatten()))
    UwithX = Us.flatten().reshape(1,-1)
    Nx = x1s.size * x2s.size
    X = XwithU[:,:Nx]
    N = X1s.size

    Xnext = A @ XwithU + B @ UwithX
    G = quadratic_regulator_cost(XwithU, UwithX)
    Jnext = np.zeros((1,N))
    Jd = np.zeros((1,Nx))

    def cost_to_go(S, X):
        return (X * (S @ X)).sum(axis=0)  # vectorized quadratic form

    def mean_bellman_residual(S, X, J_desired):
        N = J_desired.size
        err = cost_to_go(S, X) - J_desired
        loss = np.mean(err**2) # == 1/N ∑ᵢ [tr(Sxᵢxᵢ')-yᵢ]²
        # dloss_dS = 2/N ∑ᵢ errᵢ*xᵢxᵢ' = 2/N X*Diag(err)*X'
        dloss_dS = 2/N * X @ np.diag(err.reshape(-1,)) @ X.T
        return loss, dloss_dS

    S = S_guess
    eta = 0.0001
    last_loss = np.inf
    for epoch in range(1000 if running_as_notebook else 2):
        Jnext = cost_to_go(S, Xnext)
        Jd[:] = np.min((G + gamma*Jnext).reshape(us.size, Nx), axis=0)
        for i in range(10):
            loss, dloss_dS = mean_bellman_residual(S, X, Jd)
            S -= eta*dloss_dS
        clear_output(wait=True)
        print(f"epoch {epoch}: loss = {loss}, S = {S.flatten()}")
        if np.linalg.norm(last_loss - loss) < 1e-8:
            break
        last_loss = loss

    print(f"eigenvalues of S: {np.linalg.eig(S)[0]}")
    print(f"optimal S= {S_optimal.flatten()}")


FittedValueIteration(S_guess=np.eye(2), gamma=0.9)

With the initial guess of $S$ and the discount factor $\gamma$ that I've used above, the algorithm converges nicely.  You'll see that it is close to the optimal $S$, but not exactly the optimal $S$.

Interestingly, if you choose $\gamma$ to be much closer to 1, the algorithm will diverge. At some point, the sampled values over $u$ cause problems -- the Bellman equation does not actually have a steady-state solution for the discretized $u$ version of the problem (the cost-to-go is infinite).

To convince you that this is indeed the problem, here is a version that solves analytically for the optimal $u$ (given the estimated $S$).  It draws samples only over $x$, and converges beautifully to the true optimum, even when $\gamma=1$.

In [ ]:
def FittedValueIteration(S_guess, gamma=0.9):

    S_optimal = DiscreteAlgebraicRiccatiEquation(A=np.sqrt(gamma) * A,
                                                 B=B,
                                                 Q=Q,
                                                 R=R / gamma)

    x1s = np.linspace(-5,5,31)
    x2s = np.linspace(-4,4,31)
    X1s, X2s = np.meshgrid(x1s, x2s, indexing='ij')
    X = np.vstack((X1s.flatten(), X2s.flatten()))
    N = X1s.size

    Jd = np.zeros((1,N))

    def cost_to_go(S, X):
        return (X * (S @ X)).sum(axis=0)  # vectorized quadratic form

    def policy(S, X):
        return -gamma*np.linalg.inv(R + gamma*B.T @ S @ B) @ B.T @ S @ A @ X

    def mean_bellman_residual(S, X, J_desired):
        N = J_desired.size
        err = cost_to_go(S, X) - J_desired
        loss = np.mean(err**2) # == 1/N ∑ᵢ [tr(Sxᵢxᵢ')-yᵢ]²
        # dloss_dS = 2/N ∑ᵢ errᵢ*xᵢxᵢ' = 2/N X*Diag(err)*X'
        dloss_dS = 2/N * X @ np.diag(err.reshape(-1,)) @ X.T
        return loss, dloss_dS

    S = S_guess
    eta = 0.0001
    last_loss = np.inf
    for epoch in range(1000 if running_as_notebook else 2):
        U = policy(S, X)
        Xnext = A @ X + B @ U
        G = quadratic_regulator_cost(X, U)
        Jd = G + gamma*cost_to_go(S, Xnext)
        for i in range(10):
            loss, dloss_dS = mean_bellman_residual(S, X, Jd)
            S -= eta*dloss_dS
        clear_output(wait=True)
        print(f"epoch {epoch}: loss = {loss}, S = {S.flatten()}")
        if np.linalg.norm(last_loss - loss) < 1e-8:
            break
        last_loss = loss

    print(f"eigenvalues of S: {np.linalg.eig(S)[0]}")
    print(f"optimal S= {S_optimal.flatten()}")
    return S


FittedValueIteration(S_guess=np.eye(2), gamma=1);

# LQR on a Manifold

I'll also use these examples to illustrate a little more about how one might build a new model in drake (using multibody plant).  I've included the URDF in this notebook directly as a string so that you can see it and easily play with it.

## Simulating a wheel

*TODO: Consider moving this to the vehicles chapter once it takes more form*

As a warm-up, let's just make sure we understand how to simulate a wheel in Drake (using the full `MultibodyPlant` contact engine).  I've done this by putting the cart on a prismatic joint (so it can't fall down) but now adding collision geometry to the wheel and the ground.

The short story is that if you simply add a wheel (I use a sphere here) and put it into penetration with the ground, then the contact engine will implement the frictional contact which will cause the wheel to roll.  The one subtlety is that the contact is enforced at a point that is a somewhere inside the overlapping region of the wheel and the ground. (We can query it's exact location using the contact results output port of multibodyplant.)  This leads to an effective reduction in the radius of the wheel.

I've put the wheel in this model into relatively deep penetration, just to make the point.

If needed, we could have increased the friction coefficient to make sure that we are in the sticking (not sliding) regime.  But the default parameters worked just fine here.

In [ ]:
cart_with_wheel_urdf = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="CartWithWheel">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
    </collision>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="cart">
    <inertial>
      <mass value="10" />
      <inertia ixx="0" ixy="0" ixz="0" iyy="0" iyz="0" izz="0" />
    </inertial>
    <visual>
      <geometry>
        <box size=".6 .3 .3" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>
  </link>

  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="cart" />
    <origin xyz="0 0 0.25" />
    <axis xyz="1 0 0" />
  </joint>

  <link name="wheel">
    <inertial>
      <mass value="1" />
      <inertia ixx=".02" ixy="0" ixz="0" iyy="0.02" iyz="0" izz="0.02" />
    </inertial>
    <visual>
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    <collision>
      <geometry>
        <sphere radius=".05" />
      </geometry>
    </collision>
  </link>

  <joint name="wheel" type="continuous">
    <parent link="cart" />
    <child link="wheel" />
    <!-- I've place the wheel just 0.02 into penetration -->
    <origin xyz="0 0 -.22"/>  
    <axis xyz="0 1 0" />
  </joint>

  <transmission type="SimpleTransmission" name="wheel">
    <actuator name="wheel" />
    <joint name="wheel" />
  </transmission>
  
</robot>
"""

from pydrake.all import PidController

def CartWithWheelExample():
    builder = DiagramBuilder()
    time_step = 0.001

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    Parser(plant).AddModelFromString(cart_with_wheel_urdf, "urdf")
    plant.Finalize()

    kd_wheel = 1
    pid = builder.AddSystem(
        PidController(state_projection=np.array([[0, 1, 0, 0], [0, 0, 0, 1]]),
                      kp=[0],
                      ki=[0],
                      kd=[kd_wheel]))
    builder.Connect(pid.get_output_port(), plant.get_actuation_input_port())
    builder.Connect(plant.get_state_output_port(),
                    pid.get_input_port_estimated_state())
    builder.ExportInput(pid.get_input_port_desired_state(), "vd")
    builder.ExportOutput(plant.get_contact_results_output_port(),
                         "contact_results")

    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    # Setup visualization
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-2.5, xmax=2.5, ymin=-1.0, ymax=2.5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    diagram.get_input_port().FixValue(context, [0, 10])

    # Simulate
    visualizer.StartRecording(False)
    simulator.AdvanceTo(2 if running_as_notebook else 0.1)
    visualizer.PublishRecording()
    log = logger.FindLog(context)
    fig, ax = plt.subplots(1, 2, figsize=((10, 6)))
    ax[0].plot(log.sample_times(), log.data().T)
    ax[0].legend(['x', 'wheel', 'xdot', 'wheel_dot'])
    r = 0.05
    print(f"Wheel radius is {r}")
    print(f"x(tf) / wheel(tf) = {log.data()[0,-1]/log.data()[1,-1]}")
    contact_results = diagram.get_output_port().Eval(context)
    z_contact = contact_results.point_pair_contact_info(0).contact_point()[2]
    print(
        f"The contact point no-slip condition is enforced at z={z_contact}"
    )
    r_effective = r - 0.02 - z_contact
    print(f"This leads to an effective wheel radius of {r_effective}")

    F = np.array([[1, -r_effective, 0, 0], [0, 0, 1, -r_effective]])
    ax[1].plot(log.sample_times(), (F@log.data()).T)
    ax[1].legend(['x - r_effective*wheel', 'xdot - r_effective*wheel_dot'])

CartWithWheelExample()

## Balancing a 2D Segway (aka "Ballbot")

Q: Can LQR work even better if I used absolute coordinates for the bot, instead of relative? (The wheel rotates a lot, but the bot angle should not change much).

In [ ]:
ballbot_sliding_base_urdf = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="Ballbot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <drake:proximity_properties>
        <drake:mu_dynamic value="100" />
        <drake:mu_static value="100" />
      </drake:proximity_properties>
    </collision>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball_x" />

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".02" ixy="0" ixz="0" iyy="0.02" iyz="0" izz="0.02" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
    <visual> 
      <!-- add a visual cue to see when the ball is rolling -->
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <box size="0.04 .201 0.002" />
      </geometry>
      <material>
        <color rgba=".1 .1 .1 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <drake:proximity_properties>
        <drake:mu_dynamic value="100" />
        <drake:mu_static value="100" />
      </drake:proximity_properties>
    </collision>
  </link>

  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball_x" />
    <!-- height is set to (radius - 0.001) to have sufficient penetration to have a normal force that supports the required frictional force -->
    <origin xyz="0 0 .099" />
    <axis xyz="1 0 0" />
  </joint>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.018" ixy="0" ixz="0" iyy="0.018" iyz="0" izz="0.0288" />
    </inertial>
    <!-- no collision geometry since Drake AutoDiffXd doesn't support cylinder on box collisions yet. -->
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>  

  <joint name="theta_ball" type="continuous">
    <parent link="ball_x" />
    <child link="ball" />
    <axis xyz="0 -1 0" />   
    <!-- use -1 to be consistent with the planar joint having z up below -->
  </joint>

  <joint name="theta_bot" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <axis xyz="0 -1 0" />
  </joint>

  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta_bot" />
  </transmission>
  
</robot>
"""

r = 0.0995  # Effective r given the contact point (see the example above)
n = np.sqrt(1 + r**2)

F = np.array([[1, r, 0, 0, 0, 0], [0, 0, 0, 1, r, 0]])
P = np.array([[r / n, -1 / n, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0],
                [0, 0, 0, r / n, -1 / n, 0], [0, 0, 0, 0, 0, 1]])

np.testing.assert_almost_equal(P @ P.T, np.eye(4, 4))
np.testing.assert_almost_equal(P @ F.T, np.zeros((4, 2)))

def BallbotUprightState():
    state = (0, 0, 0, 0, 0, 0)
    return state

def MakeSlidingBaseBallbot(time_step=0.001):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    Parser(plant).AddModelFromString(ballbot_sliding_base_urdf, "urdf")
    plant.Finalize()

    builder.ExportInput(plant.get_actuation_input_port(), "torque")
    builder.ExportOutput(plant.get_state_output_port(), "state")
    builder.ExportOutput(scene_graph.get_query_output_port(), "query")

    return builder.Build()


def BallbotControllability(plant):
    context = plant.CreateDefaultContext()
    plant.get_input_port().FixValue(context, [0])

    context.SetDiscreteState(BallbotUprightState())

    linear = Linearize(
        plant,
        context,
        output_port_index=OutputPortSelection.kNoOutput)
    #display(Markdown("$A = " + ToLatex(linear.A()) + "$"))
    #display(Markdown("$B = " + ToLatex(linear.B()) + "$"))
    #display(
    #    Markdown("$ctrb(A,B) = " + ToLatex(ControllabilityMatrix(linear), 6)
    #             + "$"))
    display(Markdown(f"Is controllable? {IsControllable(linear)}"))

def BallbotManifoldLQR(plant):
    context = plant.CreateDefaultContext()
    plant.get_input_port().FixValue(context, [0])

    Q = np.diag((10, 10, 1, 1))
    R = np.array([1])

    context.SetDiscreteState(BallbotUprightState())

    linear = Linearize(
        plant,
        context,
        output_port_index=OutputPortSelection.kNoOutput)

    A = P @ linear.A() @ P.T
    B = P @ linear.B()

    K, S = DiscreteTimeLinearQuadraticRegulator(A, B, Q, R)

    return MatrixGain(-K @ P)

def BallbotSlidingBaseExample():
    builder = DiagramBuilder()
    time_step = 0.001

    plant = builder.AddSystem(MakeSlidingBaseBallbot(time_step))
    logger = LogVectorOutput(plant.GetOutputPort("state"), builder)

    # For completeness, we can check that the plant by itself is not controllable:
    BallbotControllability(plant)

    if False:  # Useful for debugging.
        kd_wheel = 1
        pid = builder.AddSystem(
            PidController(state_projection=np.array([[0, 1, 0, 0, 0, 0],
                                                    [0, 0, 0, 0, 1, 0]]),
                        kp=[0],
                        ki=[0],
                        kd=[kd_wheel]))
        builder.Connect(pid.get_output_port(), plant.get_input_port())
        builder.Connect(plant.GetOutputPort("state"),
                        pid.get_input_port_estimated_state())
        pid_command = builder.AddSystem(ConstantVectorSource([0, 1]))
        builder.Connect(pid_command.get_output_port(),
                        pid.get_input_port_desired_state())
    else:
        controller = builder.AddSystem(BallbotManifoldLQR(plant))
        controller.set_name("LQR Controller")
        builder.Connect(plant.GetOutputPort("state"),
                        controller.get_input_port())
        builder.Connect(controller.get_output_port(),
                        plant.get_input_port())

    # Setup visualization
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-.5, xmax=.5, ymin=-.2, ymax=.5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder,
                                                   plant.GetOutputPort("query"),
                                                   meshcat)

    diagram = builder.Build()

    # For reference, let's draw the diagram we've assembled:
    #display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].#create_svg()))

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    #simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    context = simulator.get_mutable_context()
    x0 = BallbotUprightState() + P.T @ (0.0 * np.random.randn(4,)
                                        + np.array([1, 0, 0, 0]))
    assert np.allclose(F @ x0, np.zeros(2))
    context.SetDiscreteState(x0)

    # Simulate

    visualizer.StartRecording(False)
    simulator.AdvanceTo(4 if running_as_notebook else 0.1)
    visualizer.PublishRecording()
    log = logger.FindLog(context)
    fig, ax = plt.subplots(1, 2, figsize=((10, 6)))
    ax[0].plot(log.sample_times(), log.data()[:3,:].T)
    ax[0].legend(['x', 'theta_ball', 'theta_bot'])
    ax[1].plot(log.sample_times(), (F@log.data()).T)
    ax[1].legend(['x - r*theta_ball', 'xdot - r*thetadot_ball'])

BallbotSlidingBaseExample()


But there is subtle point to be made here.  I've modeled the wheels as a prismatic joint at the base.  Let's instead model the robot with a floating joint, and using collisions elements to simulate the interaction with the robot and the ground...

In [ ]:
ballbot_floating_base_urdf = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="Ballbot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
    </collision>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".02" ixy="0" ixz="0" iyy="0.02" iyz="0" izz="0.02" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
    <visual> 
      <!-- add a visual cue to see when the ball is rolling -->
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <box size="0.04  0.002 .201" />
      </geometry>
      <material>
        <color rgba=".1 .1 .1 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
    </collision>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.018" ixy="0" ixz="0" iyy="0.018" iyz="0" izz="0.0288" />
    </inertial>
    <collision>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
    </collision>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <drake:joint name="floating_base" type="planar">
    <parent link="world" />
    <child link="ball" />
    <origin rpy="1.57 0 0" xyz="0 0 .1" />
  </drake:joint>

  <joint name="theta2" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin rpy="-1.57 0 0" xyz="0 0 0" />
    <axis xyz="0 -1 0" />
  </joint>

  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta2" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

</robot>
"""

def BallbotFloatingBaseExample():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    Parser(plant).AddModelFromString(ballbot_floating_base_urdf, "urdf")
    plant.Finalize()

    # Just use zero instead of a controller to start
    command = builder.AddSystem(ConstantVectorSource([0.0]))
    builder.Connect(command.get_output_port(), plant.get_actuation_input_port())

    # Setup visualization
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-.5, xmax=.5, ymin=-.2, ymax=.5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    context = simulator.get_mutable_context()

    State = namedview("State", [
        "x", "z", "theta1", "theta2", "xdot", "zdot", "theta1dot", "theta2dot"
    ])
    x0 = State(np.zeros(8))
    x0.z = .15
    x0.theta1 = 0.05
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositionsAndVelocities(plant_context, x0[:])

    # Simulate
    visualizer.StartRecording(False)
    simulator.AdvanceTo(3 if running_as_notebook else 0.1)
    visualizer.PublishRecording()
    xf = State(plant.GetPositionsAndVelocities(plant_context))
    print(f"z at final time = {xf.z}")

BallbotFloatingBaseExample()

Here's the real test... you should be able to run the controller designed for the simple model on the floating base version of the ballbot.  Give it a shot!

In [ ]:
def BallbotFloatingBaseLqrExample():
    builder = DiagramBuilder()
    time_step = 0.001
    ballbot_minimal_coordinates = builder.AddSystem(
        MakeSlidingBaseBallbot(time_step))

    controller_minimal_coordinates = builder.AddSystem(
        BallbotManifoldLQR(ballbot_minimal_coordinates))
    controller_minimal_coordinates.set_name("LQR Controller for Minimal")
    builder.Connect(controller_minimal_coordinates.get_output_port(),
                    ballbot_minimal_coordinates.get_input_port())
    builder.Connect(
        ballbot_minimal_coordinates.GetOutputPort("state"),
        controller_minimal_coordinates.get_input_port())

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    Parser(plant).AddModelFromString(ballbot_floating_base_urdf, "urdf")
    plant.Finalize()

    controller = builder.AddSystem(
        BallbotManifoldLQR(ballbot_minimal_coordinates))
    controller.set_name("LQR Controller for Floating")
    builder.Connect(controller.get_output_port(),
                    plant.get_actuation_input_port())
    #builder.ExportInput(plant.get_actuation_input_port(), "u")

    MinimalState = namedview("MinimalState", ["x", "theta_ball", "theta_bot_m_ball", "xdot", "thetadot_bot", "thetadot_bot_m_ball"])
    FloatingState = namedview("FloatingState", [
        "x", "z", "theta_ball", "theta_bot_m_ball", "xdot", "zdot", "thetadot_ball", "thetadot_bot_m_ball"
    ])

    m_ind = MinimalState(range(6))
    f_ind = FloatingState(range(8))
    P_floating_to_minimal = np.delete(np.eye(8), [f_ind.z, f_ind.zdot], 0)
    floating_state_to_minimal_state = builder.AddSystem(
        MatrixGain(P_floating_to_minimal))
    builder.Connect(plant.get_state_output_port(),
                    floating_state_to_minimal_state.get_input_port())
    builder.Connect(floating_state_to_minimal_state.get_output_port(),
                    controller.get_input_port())

    # Setup visualization
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-.5, xmax=.5, ymin=-.2, ymax=.5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph,
                                                   meshcat)

    minimal_state_logger = LogVectorOutput(
        ballbot_minimal_coordinates.GetOutputPort("state"), builder)
    floating_state_logger = LogVectorOutput(plant.get_state_output_port(),
                                            builder)
    minimal_control_logger = LogVectorOutput(
        controller_minimal_coordinates.get_output_port(), builder)
    floating_control_logger = LogVectorOutput(controller.get_output_port(),
                                              builder)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    #simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    context = simulator.get_mutable_context()
    #diagram.GetInputPort("u").FixValue(context, [3])

    x0 = FloatingState(np.zeros(8))
    x0.z = -0.0018  # the approximate resting depth due to soft contact
    x0.theta_bot_m_ball = 0.05
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositionsAndVelocities(plant_context, x0[:])

    ballbot_minimal_state_context = ballbot_minimal_coordinates.GetMyMutableContextFromRoot(
        context)
    minimal_x0 = MinimalState(P_floating_to_minimal @ x0[:])
    ballbot_minimal_state_context.SetDiscreteState(minimal_x0[:])

    values = plant.AllocateDiscreteVariables()
    plant.CalcDiscreteVariableUpdates(plant_context, values)
    print(f"floating_xn = {FloatingState(values.get_value())}")
    print(
        f"projected_xn = {MinimalState( P_floating_to_minimal @ values.get_value())}"
    )

    values = ballbot_minimal_coordinates.AllocateDiscreteVariables()
    ballbot_minimal_coordinates.CalcDiscreteVariableUpdates(
        ballbot_minimal_state_context, values)
    print(f"minimal_xn = {MinimalState(values.get_value())}")

    # Simulate
    visualizer.StartRecording(False)
    simulator.AdvanceTo(5 if running_as_notebook else 0.1)
    visualizer.PublishRecording()

    minimal_state_log = minimal_state_logger.FindLog(context)
    floating_state_log = floating_state_logger.FindLog(context)
    fig, ax = plt.subplots(1,2,figsize=(15,8))
    ax[0].plot(minimal_state_log.sample_times(), minimal_state_log.data().T)
    ax[0].set_title('minimal state')
    ax[0].legend([
        "x", "theta_ball", "theta_bot_m_ball", "xdot", "thetadot_ball",
        "thetadot_bot_m_ball"
    ])
    if True:  # plot projection of floating state
        ax[1].plot(floating_state_log.sample_times(),
            (P_floating_to_minimal @ floating_state_log.data()).T)
    else:
        ax[1].plot(floating_state_log.sample_times(),
                floating_state_log.data().T)
        ax[1].legend([ "x", "z", "theta1", "theta2", "xdot", "zdot", "theta1dot", "theta2dot"])
    ax[1].set_title('floating state')
    ax[0].set_ylim(ax[1].get_ylim())

BallbotFloatingBaseLqrExample()